# Инструмент для парсинга тг-чата

В данном случае под парсингом подразумевается работы с выгруженным архивом тг-чата в формате json. Получить архив можно следующим образом:

1) Устанавливаем десктопную версию тг по ссылке: https://desktop.telegram.org/
2) После установки логинимся и открываем нужный нам чат
3) Кликаем на три точки в верхнем правом углу чата
4) В меню выбираем "Экспорт истории чата"
5) После этого в окошке выбираем настройки экспорта: типы данных для выгрузки и временной период. Если вас интересует только текст чата, то смысла отмечать галочкой для скачивания изображения, видео, стикеры, аудио и т.д. нет
6) Запускаем процесс экспорта и ждем. Процесс не быстрый, но окно можно свернуть и делать что-то параллельно
7) Наконец, получаем заветный архив тг-чата

Внутри архива будет много различных папок и html-файлы "messages.html", "messages2.html" и т.д. В контексте анализа текстов сообщения нас интересуют именно эти html-файлы, остальное не нужно. Копируем эти файлы в папку, в которой расположен ipynb-файл этого инструмента и начинаем анализ.

Если в процессе работы у тебя возникнут предложения по улучшению инструмента, можно написать мне в тг: @esthesuntik

### Импортируем библиотеки и модули

In [1]:
from bs4 import BeautifulSoup as BS
import codecs
import docx

### Поиск сообщений конкретного автора

Ниже написана функция, с помощью которой происходит обработка json-файла и извлечение текстов сообщений автора. В ней ничего не нужно менять, можно просто изучить в связке с одним из html-файлов "messages". Затем запустить остальные ячейки.

In [65]:
# на вход функция принимает название анализируемого файла и имя / ник автора, сообщения которого мы ищем
def messages_finder(file_name, search_name):

    # считываем json-файл
    file = codecs.open(file_name, 'r', 'utf-8')
    html = file.read()

    soup = BS(html, "lxml")

    # получаем все сообщения из чата в формате списка
    all_messages = soup.find_all('div', class_ = 'body')

    author_messages = []

    # в цикле ищем сообщения с нужным автором
    for i in range(len(all_messages)):
        author = all_messages[i].find('div', class_ = 'from_name')
        
        if author != None:
            if author.text.strip() == search_name:
                author_messages.append(all_messages[i])
        else:
            n = i-1
            author = all_messages[n].find('div', class_ = 'from_name')
            
            while author == None:
                n = n-1
                author = all_messages[n].find('div', class_ = 'from_name')
    
            if author.text.strip() == search_name:
                author_messages.append(all_messages[i])

    
    # из кода сообщений нужного автора собираем тексты в список
    author_messages_texts = []

    for message in author_messages:
        forward_check = message.find('div', class_ = 'forwarded')

        if forward_check == None:
            message_text = message.find('div', class_ = 'text')
        
            if message_text != None:
                author_messages_texts.append(message_text.text)

    # функция возвращает список с текстами сообщений
    return author_messages_texts

В ячейку ниже нужно вписать все названия html-файлов из архива (которые мы в начале скопировали в папку к инструменту).

Решение костыльное и в будущем будет исправлено на более удобный вариант.

In [ ]:
files = ['messages.html', 'messages2.html', 'messages3.html', 'messages4.html', 'messages5.html', 'messages6.html',
         'messages7.html', 'messages8.html', 'messages9.html', 'messages10.html', 'messages11.html', 'messages12.html',
         'messages13.html', 'messages14.html', 'messages15.html']

В переменную ниже нужно вписать реальное имя пользователя (как он отображается у вас в тг).

Важно, чтобы в начале и в конце имени не было лишних пробелов

In [ ]:
search_name = 'Имя пользователя'

In [ ]:
# прогоняем через функцию все html-файлы тг-архива

messages_texts = []
all_messages_count = 0

for file in files:
    file_messages = messages_finder(file, search_name)
    messages_texts += file_messages
    
    all_messages_count += len(file_messages)

В ячейке ниже 'file_name' замените на удобное название файла

In [ ]:
# cохраняем сообщения в docx-файл

document = docx.Document()

for message_text in messages_texts:

    document.add_paragraph(message_text)

document.save('file_name.docx')

### Парсинг сообщений всех авторов

Функция ниже является упрощенной версией алогритма по поиску сообщения конкретного автора. Тут мы не фильтруем поиск по конкретному имени и собираем все тексты сообщений.

In [2]:
def messages_parser(file_name):
    file = codecs.open(file_name, 'r', 'utf-8')
    html = file.read()

    soup = BS(html, "lxml")

    all_messages = soup.find_all('div', class_ = 'text')

    all_messages_texts = []

    for message in all_messages:
        forward_check = message.find('div', class_ = 'forwarded')

        if forward_check == None:
            all_messages_texts.append(message.text)

    return(all_messages_texts)

В ячейку ниже нужно вписать все названия html-файлов из архива (которые мы в начале скопировали в папку к инструменту).

Решение костыльное и в будущем будет исправлено на более удобный вариант.

In [7]:
files = ['messages.html', 'messages2.html', 'messages3.html', 'messages4.html', 'messages5.html', 'messages6.html',
         'messages7.html', 'messages8.html', 'messages9.html', 'messages10.html', 'messages11.html', 'messages12.html',
         'messages13.html', 'messages14.html', 'messages15.html']

In [ ]:
# прогоняем через функцию все html-файлы тг-архива

messages_texts = []

for file in files:
    file_messages = messages_parser(file)
    messages_texts += file_messages

В ячейке ниже 'file_name' замените на удобное название файла

In [ ]:
document = docx.Document()

for message_text in messages_texts:

    document.add_paragraph(message_text)

document.save('file_name.docx')

Ура, тексты собраны! Теперь можно обработать их. Например, с помощью другого моего инструмента по анализу текста: [ссылка на код](https://github.com/alsosha/text_analysis/blob/main/text_analysis.ipynb)

А потом можно сгенерировать облако слов: [ссылка на код](https://github.com/alsosha/word_cloud_generator/blob/main/word_cloud_generator.ipynb). Ну или построить любые другие графики. Удачи :)